In [2]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential, load_model
from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *

Using TensorFlow backend.


In [3]:
print(tf.test.is_gpu_available())

True


In [348]:
def sanitize(string):
    words = string.split(' ')
    return words


def converter(x):
    try:
        return ' '.join([x.lower() for x in str(x).split() if x not in stop_words])
    except AttributeError:
        return None  # or some other value
    
def whole(x):
    return int(round(x))

whole(1.0)




'ff  ffa'

In [5]:
#read in the csv files 

quora = pd.read_csv("quora_duplicate_questions (1).tsv", sep='\t')


quora['question1']=quora['question1'].fillna("")
quora['question2']=quora['question2'].fillna("")


quora['title1_tokenized'] = \
    quora.loc[:, 'question1'] \
         .apply(sanitize)
quora['title2_tokenized'] = \
    quora.loc[:, 'question2'] \
         .apply(sanitize)



quora.head()

,id,qid1,qid2,question1,question2,is_duplicate,title1_tokenized,title2_tokenized
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[What, is, the, step, by, step, guide, to, inv...","[What, is, the, step, by, step, guide, to, inv..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[What, is, the, story, of, Kohinoor, (Koh-i-No...","[What, would, happen, if, the, Indian, governm..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[How, can, I, increase, the, speed, of, my, in...","[How, can, Internet, speed, be, increased, by,..."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[Why, am, I, mentally, very, lonely?, How, can...","[Find, the, remainder, when, [math]23^{24}[/ma..."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[Which, one, dissolve, in, water, quikly, suga...","[Which, fish, would, survive, in, salt, water?]"


In [28]:
Tb = Table()
list1 = []
list2 = []

with open("quora_duplicate_questions (1).tsv", 'r', encoding='utf-8') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(6):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        Tb=Tb.with_columns(list1[0][ele], list2[ele])


Tb.show(5)

id,qid1,qid2,question1,question2,is_duplicate
0,1,2,What is the step by step guide to invest in share market ...,What is the step by step guide to invest in share market?,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Koh ...,0
2,5,6,How can I increase the speed of my internet connection w ...,How can Internet speed be increased by hacking through DNS?,0
3,7,8,Why am I mentally very lonely? How can I solve it?,Find the remainder when [math]23^{24}[/math] is divided ...,0
4,9,10,"Which one dissolve in water quikly sugar, salt, methane ...",Which fish would survive in salt water?,0


In [41]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [311]:
import nltk
from nltk.corpus import stopwords
stop_words=(stopwords.words('english'))
stop_words=np.append(stop_words, "i'm")
print(stop_words)

['i' 'me' 'my' 'myself' 'we' 'our' 'ours' 'ourselves' 'you' "you're"
 "you've" "you'll" "you'd" 'your' 'yours' 'yourself' 'yourselves' 'he'
 'him' 'his' 'himself' 'she' "she's" 'her' 'hers' 'herself' 'it' "it's"
 'its' 'itself' 'they' 'them' 'their' 'theirs' 'themselves' 'what' 'which'
 'who' 'whom' 'this' 'that' "that'll" 'these' 'those' 'am' 'is' 'are'
 'was' 'were' 'be' 'been' 'being' 'have' 'has' 'had' 'having' 'do' 'does'
 'did' 'doing' 'a' 'an' 'the' 'and' 'but' 'if' 'or' 'because' 'as' 'until'
 'while' 'of' 'at' 'by' 'for' 'with' 'about' 'against' 'between' 'into'
 'through' 'during' 'before' 'after' 'above' 'below' 'to' 'from' 'up'
 'down' 'in' 'out' 'on' 'off' 'over' 'under' 'again' 'further' 'then'
 'once' 'here' 'there' 'when' 'where' 'why' 'how' 'all' 'any' 'both'
 'each' 'few' 'more' 'most' 'other' 'some' 'such' 'no' 'nor' 'not' 'only'
 'own' 'same' 'so' 'than' 'too' 'very' 's' 't' 'can' 'will' 'just' 'don'
 "don't" 'should' "should've" 'now' 'd' 'll' 'm' 'o' 're' 've' 'y'

In [157]:
stop_words_punct=np.append(stop_words, "?")


In [44]:

from scipy import spatial

def cos_sim(vec1, vec2):
    return (1 - spatial.distance.cosine(vec1, vec2))

In [48]:
import difflib

case_a = 'i like apple'
case_b = 'i like pie'

output_list = [li for li in difflib.ndiff(case_a, case_b) if li[0] != ' ']

In [238]:
from collections import deque

In [366]:
def diff(sen1, sen2):   
    a=[]
    b=[]
    c=[]
    wordlist1 = sen1.split()
    wordlist2 = sen2.split()   
    for i in wordlist1:
        if i.lower() in stop_words:  #omit stop words
            continue    
        if i not in wordlist2:
            a=np.append(a, i)
        else:
            c=np.append(c, i)
    for j in wordlist2:
        if j.lower() in stop_words:   #omit stop words
            continue
        if j not in wordlist1:
            b=np.append(b, j)  
    return np.array(a),np.array(b),np.array(c)       


def seqDiff(sen1,sen2,a,b):      #include length 1
    #wordlist1 = sen1.split()
    #wordlist2 = sen2.split()
    sen1=[x.lower() for x in sen1]
    sen2=[x.lower() for x in sen2]
    
    
    if len(sen1)==1:
                
        if sen1[0] in sen2 or sen1[0] in stop_words:
            if len(a)>0:
                b.append(a)
                
            return b
        
        else:
            a=np.append(a,sen1[0])
            b.append(a)
                
            return b
    
    elif sen1[0] in sen2:
        
        if len(a)>0:
            b.append(a)
        a=[]        
        return seqDiff(sen1[1:], sen2,a,b)
        
    elif (sen1[0] not in sen2):
        
        if sen1[0] not in stop_words:
            a=np.append(a,sen1[0])
        
        elif len(a)>0: 
            b.append(a)
            a=[]
            
        return seqDiff(sen1[1:], sen2,a,b)
    
    
    
def seqDiff2(sen1,sen2,a,b):     # inlude length 1
    #wordlist1 = sen1.split()
    #wordlist2 = sen2.split()
    
    if len(sen1)==1:
        
        if  sen1[0] in sen2.lower() or sen1[0].lower() in sen2.lower() or sen1[0] in sen2 or sen1[0].lower() in sen2 or sen1[0].lower() in stop_words:
            if len(a)!=0:
                b.append(a)
            return b
        if sen1[0] not in sen2:
            a=np.append(a,sen1[0])
            b.append(a)
                
            return b
    
    elif sen1[0] in sen2 or sen1[0].lower() in sen2:
        if len(a)!=0:
            b.append(a)
            a=[]
                
        return seqDiff(sen1[1:], sen2,a,b)
        
    elif (sen1[0] not in sen2) and (sen1[0].lower() not in sen2):
        
        if sen1[0].lower() not in stop_words:
            a=np.append(a,sen1[0])
        
        return seqDiff(sen1[1:], sen2,a,b)
                

In [344]:
g="How can I be a good geologist?" 
hue="What should I do to be a great geologist"
g=g.split()
hue=hue.split()
deck=deque()
temp=seqDiff(g,hue,[],deck)
temp


AttributeError: 'list' object has no attribute 'replace'

In [218]:
hh=[1,2,3,4]
hhh=[12,3,4]
hhhh=np.append(hh,hhh)
hhhh

array([ 1,  2,  3,  4, 12,  3,  4])

In [49]:
output_list

['- a', '+ i', '- p', '- l']

In [301]:
true_Tb = Tb.where("is_duplicate", are.equal_to("1"))
true_Tb.show(30)

id,qid1,qid2,question1,question2,is_duplicate
5,11,12,Astrology: I am a Capricorn Sun Cap moon and cap rising. ...,"I'm a triple Capricorn (Sun, Moon and ascendant in Capri ...",1
7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1
15,31,32,What would a Trump presidency mean for current internati ...,How will a Trump presidency affect the students presentl ...,1
16,33,34,What does manipulation mean?,What does manipulation means?,1
18,37,38,Why are so many Quora users posting questions that are r ...,Why do people ask Quora questions which can be answered ...,1
20,41,42,Why do rockets look white?,Why are rockets and boosters painted white?,1
29,59,60,How should I prepare for CA final law?,How one should know that he/she completely prepare for C ...,1


In [368]:
row = true_Tb.num_rows
sent1 =  true_Tb.column("question1")
sent2 = true_Tb.column("question2")
a=[0]*row
b=[0]*row
c=[0]*row
bleh=deque()

for x in range((row)):
    sen1=no_ques(sent1[x])
    sen2=no_ques(sent2[x])
    sen1=sen1.split()
    sen2=sen2.split()
    
    d = seqDiff(sen1, sen2, bleh, [])
    e = seqDiff(sen2,sen1,bleh,[])
    a[x]=d
    b[x]=e    
   



In [367]:
def no_ques(sent1):
    
    g = [x.replace("?", "") for x in sent1]
    
    return ''.join(g)

In [369]:
new_Tb=Table().with_column( "sent1_only", a, "sent2_only",b)

In [ ]:
new_Tb.to_csv("sanitized_QQP_dataset.csv")

In [370]:
new_Tb.show(40)  #omitting stopwords

sent1_only,sent2_only
"[array(['astrology:'], dtype='<U32'), array(['sun', 'cap ...","[array(['triple'], dtype='<U32'), array(['(sun,'], dtype ..."
"[array(['good'], dtype='<U32')]","[array(['great'], dtype='<U32')]"
"[array(['read'], dtype='<U32'), array(['find'], dtype='< ...","[array(['see'], dtype='<U32')]"
[],[]
"[array(['like'], dtype='<U32')]",[]
"[array(['would'], dtype='<U32'), array(['mean'], dtype=' ...","[array(['affect'], dtype='<U32'), array(['presently'], d ..."
"[array(['mean'], dtype='<U32')]","[array(['means'], dtype='<U32')]"
"[array(['many'], dtype='<U32'), array(['users', 'posting ...","[array(['people', 'ask'], dtype='<U32'), array(['easily' ..."
"[array(['look'], dtype='<U32')]","[array(['boosters', 'painted'], dtype='<U32')]"
"[array(['law'], dtype='<U32')]","[array(['one'], dtype='<U32'), array(['know'], dtype='<U ..."


In [38]:
true_Tb.to_csv("QQP_Trues")

In [7]:
#create a dictionary

MAX_NUM_WORDS = 20000
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=MAX_NUM_WORDS)

corpus_x1 = quora.title1_tokenized
corpus_x2 = quora.title2_tokenized
corpus = pd.concat([
  corpus_x1, corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(corpus)
x1 = tokenizer \
    .texts_to_sequences(corpus_x1)
x2 = tokenizer \
    .texts_to_sequences(corpus_x2)

word_index=tokenizer.word_index




corpus.head()

0    [What, is, the, step, by, step, guide, to, inv...
1    [What, is, the, story, of, Kohinoor, (Koh-i-No...
2    [How, can, I, increase, the, speed, of, my, in...
3    [Why, am, I, mentally, very, lonely?, How, can...
4    [Which, one, dissolve, in, water, quikly, suga...
dtype: object

In [8]:
max_seq_len = max([            #why does seq length change when I change dictionary size?
    len(seq) for seq in x1])
max_seq_len

115

In [9]:
max_seq_len1 = max([
    len(seq) for seq in x1])
print(max_seq_len1)   

MAX_SEQUENCE_LENGTH = 115  #better to have words covered than uncovered
x1 = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2 = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x1[:5]




115


array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     2,     3,     1,  1394,    54,  1394,  2940,
            7,   519,     8,   748,   566,     8,    58],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     

In [10]:


dup = quora.is_duplicate

y = keras \
    .utils \
    .to_categorical(dup)



print(len(quora))

y[:8]

404290


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [11]:
import math


def foo():
    q=[]
    p=0
    for x in quora.is_duplicate:
        if math.isnan(x):
            q = np.append(q, p)
        p+=1
    return q
foo()

[]

In [12]:
# We randomly select 5,000 paraphrases and 5,000 non-paraphrases as the dev set, and sample
#another 5,000 paraphrases and 5,000 non-paraphrases as the
#test set. We keep the remaining instances as the training set 2

label = quora.is_duplicate
eq_list = []
uneq_list=[]

count = 0

for x in range(len(quora.is_duplicate)):
    if label[x]==1:
        eq_list = np.append(eq_list, x) 
    
    else:
        uneq_list = np.append(uneq_list, x)

        
eq_list[:3]



array([ 5.,  7., 11.])

In [13]:
eq_list=eq_list.astype(int)
uneq_list=uneq_list.astype(int)

In [14]:
import random

random.shuffle(eq_list)
random.shuffle(uneq_list)

eq_list[:10]


array([316085, 199179,  30607,  49563, 306949, 348491,  65560, 308090,
       191864,  79564])

In [15]:
index_test_eq = eq_list[:5000]

index_test_uneq = uneq_list[:5000]

index_dev_eq = eq_list[5000:10000]

index_dev_uneq = uneq_list[5000:10000]

index_train_eq = eq_list[10000:]

index_train_uneq = uneq_list[10000:]

index_test = np.append(index_test_eq, index_test_uneq)

index_dev = np.append(index_dev_eq, index_dev_uneq)

index_train = np.append(index_train_eq, index_train_uneq)

random.shuffle(index_test)
random.shuffle(index_dev)
random.shuffle(index_train)



In [16]:


x1_test = [x1[i] for i in index_test]

x2_test = [x2[i] for i in index_test]

y_test = [y[i] for i in index_test]


x1_dev = [x1[i] for i in index_dev]

x2_dev = [x2[i] for i in index_dev]

y_dev=[y[i] for i in index_dev]


x1_train = [x1[i] for i in index_train]

x2_train = [x2[i] for i in index_train]

y_train=[y[i] for i in index_train]


print("Training Set")
print("-" * 10)
print(np.shape(x1_train))
print(np.shape(x2_train))
print(np.shape(y_train))


print("-" * 10)
print(np.shape(x1_test))
print(np.shape(x2_test))
print(np.shape(y_test))


print("Test Set")
print(y_test[:5])

Training Set
----------
(384290, 115)
(384290, 115)
(384290, 2)
----------
(10000, 115)
(10000, 115)
(10000, 2)
Test Set
[array([1., 0.], dtype=float32), array([1., 0.], dtype=float32), array([0., 1.], dtype=float32), array([0., 1.], dtype=float32), array([0., 1.], dtype=float32)]


In [17]:
#add cos similarity value to last elem of word embedding
x1_test_cos=[]
x2_test_cos=[]
x1_train_cos=[]
x2_train_cos=[]
x1_dev_cos=[]
x2_dev_cos=[]

from scipy import spatial

def cos_sim(vec1, vec2):
    return (1 - spatial.distance.cosine(vec1, vec2))

for x in range(len(y_train)):
    
    
    cos_train=cos_sim(x1_train[x], x2_train[x])
    
    
    x1_train_cos.append(np.append(x1_train[x], cos_train))
    x2_train_cos.append(np.append(x2_train[x], cos_train))
    
    


for x in range(len(y_test)):
    cos_dev=cos_sim(x1_dev[x], x2_dev[x])
    cos_test=cos_sim(x1_test[x], x2_test[x])
    x1_test_cos.append(np.append(x1_test[x], cos_test))
    x2_test_cos.append(np.append(x2_test[x], cos_test))
    x1_dev_cos.append(np.append(x1_dev[x], cos_dev))
    x2_dev_cos.append(np.append(x2_dev[x], cos_dev))

/usr/local/lib/python3.5/dist-packages/scipy/spatial/distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [21]:
np.array(x1_test_cos)[:10]
len(x1_train_cos[1])

116

In [25]:
#BiLSTM with GLoVE 


NUM_CLASSES=2 #boolean; 1 or 0

MAX_NUM_WORDS = 20000 #rough estimate

MAX_SEQUENCE_LENGTH = 116 #how long one sentence is

NUM_EMBEDDING_DIM = 300 #this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension




top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')



embedding_layer = embedding_layer = Embedding(len(word_index)+1,
                            NUM_EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = Bidirectional(LSTM(NUM_LSTM_UNITS))
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 1024

NUM_EPOCHS = 15

earlyStopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')
mcp_save = ModelCheckpoint('quora_weights_with_GLoVE.h5', verbose=1, save_best_only=True, monitor='val_loss', mode='min')


Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 116)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 116)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 116, 300)     60332400    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None

In [ ]:
#BiLSTM with last cos_sim element concatenated?
NUM_CLASSES=2 #boolean; 1 or 0

#rough estimate

MAX_SEQUENCE_LENGTH = 51 #how long one sentence is

#this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = 256

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')



embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)



shared_lstm = Bidirectional(LSTM(NUM_LSTM_UNITS))

top_output = shared_lstm(top_embedded)

bm_output = shared_lstm(bm_embedded)




merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 30

earlyStopping = EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='max')
mcp_save = ModelCheckpoint('BiLSTM_PAWS_QQP_concat_sim_cos.h5', save_best_only=True,  verbose=1, monitor='val_acc', mode='max')


In [26]:

history = model.fit(x=([np.array(x1_train_cos), np.array(x2_train_cos)]), y=np.array(y_train), batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=1, callbacks=[earlyStopping, mcp_save], validation_data=([np.array(x1_dev_cos), np.array(x2_dev_cos)], np.array(y_dev)),
    
    shuffle=True
)

Instructions for updating:
Use tf.cast instead.
Train on 384290 samples, validate on 10000 samples
Epoch 1/15
384290/384290 [==============================] - 302s 786us/step - loss: 0.5586 - acc: 0.7119 - val_loss: 0.5984 - val_acc: 0.6803

Epoch 00001: val_loss improved from inf to 0.59837, saving model to quora_weights_with_GLoVE.h5
Epoch 2/15
384290/384290 [==============================] - 288s 749us/step - loss: 0.5126 - acc: 0.7474 - val_loss: 0.5627 - val_acc: 0.7122

Epoch 00002: val_loss improved from 0.59837 to 0.56267, saving model to quora_weights_with_GLoVE.h5
Epoch 3/15
384290/384290 [==============================] - 268s 698us/step - loss: 0.4926 - acc: 0.7602 - val_loss: 0.5487 - val_acc: 0.7209

Epoch 00003: val_loss improved from 0.56267 to 0.54866, saving model to quora_weights_with_GLoVE.h5
Epoch 4/15
384290/384290 [==============================] - 268s 697us/step - loss: 0.4759 - acc: 0.7709 - val_loss: 0.5332 - val_acc: 0.7316

Epoch 00004: val_loss improved fr

In [20]:

history = model.fit(x=([np.array(x1_train), np.array(x2_train)]), y=np.array(y_train), batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=1, callbacks=[earlyStopping, mcp_save], validation_data=([np.array(x1_dev), np.array(x2_dev)], np.array(y_dev)),
    
    shuffle=True
)

Train on 384290 samples, validate on 10000 samples
Epoch 1/15
384290/384290 [==============================] - 301s 783us/step - loss: 0.5399 - acc: 0.7276 - val_loss: 0.5748 - val_acc: 0.6979
Epoch 2/15
384290/384290 [==============================] - 300s 779us/step - loss: 0.5068 - acc: 0.7511 - val_loss: 0.5521 - val_acc: 0.7143
Epoch 3/15
384290/384290 [==============================] - 299s 777us/step - loss: 0.4878 - acc: 0.7637 - val_loss: 0.5325 - val_acc: 0.7257
Epoch 4/15
384290/384290 [==============================] - 303s 789us/step - loss: 0.4697 - acc: 0.7747 - val_loss: 0.5322 - val_acc: 0.7318
Epoch 5/15
384290/384290 [==============================] - 295s 769us/step - loss: 0.4547 - acc: 0.7835 - val_loss: 0.5230 - val_acc: 0.7360
Epoch 6/15
384290/384290 [==============================] - 302s 785us/step - loss: 0.4395 - acc: 0.7930 - val_loss: 0.5301 - val_acc: 0.7397
Epoch 7/15
384290/384290 [==============================] - 303s 789us/step - loss: 0.4250 - acc:

In [23]:
model = load_model("quora_weights_with_GLoVE.h5")



In [25]:
predicts = model.predict(
    [x1_test, x2_test])

array([1., 0.], dtype=float32)

In [26]:
one = Tb.column('question1')
two = Tb.column('question2')
no = []
err1 = []
err2 = []
wrongCat = []


for x in range(len(index_test)):
    if (np.argmax(predicts[x])!=(np.argmax(y_test[x]))):
        no=np.append(no, index_test[x])
        err1=np.append(err1, one[index_test[x]])
        err2=np.append(err2, two[index_test[x]])
        wrongCat = np.append(wrongCat, np.argmax(y_test[x]))
        
print(1-(len(wrongCat)/len(x1_test)))

0.7455


In [27]:
error_analysis = Table()


error_analysis=error_analysis.with_column("id", no, "sent1", err1, "sent2", err2, "classification", wrongCat)
error_analysis.show(30)
           


id,sent1,sent2,classification
117573,What was your favourite subject at school and why?,What is/was your favourite subject at school and why?,1
357893,Suppose Host A sends two TCP segments back to back to Ho ...,Suppose Host A sends two TCP segments back to back to Ho ...,1
221481,Will I always find chess easier than snooker?,Why do I find chess easier than snooker?,0
187484,What is the career of photography?,What's necessary for a successful career in photography?,1
85852,Could we create an inmortal and indestructible brain usi ...,Could we create an indestructible and inmortal brain wit ...,1
307703,What are good youtube channels that explain informative ...,What are the useful YouTube channels to gain knowledge?,0
182829,Will I be in trouble if I download movies from torrents ...,In present days download movies from torrent is a crimin ...,1
127301,What is the best way to retain what you learn?,What are the best ways to improve my memory?,0
278486,Why doesn't the police or the government help the child ...,Why doesn't the police or the government take actions ag ...,1
203535,In what ways is Australia better or different than America?,How is living in Australia different from living in the ...,1


In [133]:
hue = error_analysis.where("classification", are.equal_to(0)) 
hue


id,sent1,sent2,classification
23,Does marijuana make you gain or lose weight?,Does marijuana make you lose weight?,0
46,How can I learn online trading in india?,How can I do online trading in india?,0
51,What if the 1802 Peace of Amiens would have been upheld?,What is the best thing you have ever done to achieve inn ...,0
53,"What are some overrated firearms, and why?",Which historical figure is overrated?,0
95,Pauls pemdulum theory?,"With the modern technology, what are some impending dang ...",0
114,How do I cope with major depression?,How do you cope with depression?,0
220,Is PESIT South campus mechanical really that bad?,Is PESIT south campus really bad?,0
307,Who is likely to become the next President of India?,What should I do to become the next president of India?,0
324,How can India and China be friends?,Should India join hands with China?,0
397,How do I improve leadreship skills?,How can we improve learning skills?,0


In [24]:


print('Found %s word vectors.' % len(glove))

NUM_EMBEDDING_DIM = 300

embedding_matrix = np.zeros((len(word_index)+1, NUM_EMBEDDING_DIM))


for word, i in word_index.items():
    embedding_vector = glove.get(word)
    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Found 2195895 word vectors.


In [23]:
def process_glove_line(line, dim):
    word = None
    embedding = None

    try:
        splitLine = line.split()
        word = " ".join(splitLine[:len(splitLine)-dim])
        embedding = np.array([float(val) for val in splitLine[-dim:]])
    except:
        print(line)

    return word, embedding

def load_glove_model(glove_filepath, dim):
    with open(glove_filepath, encoding="utf8" ) as f:
        content = f.readlines()
        model = {}
        for line in content:
            word, embedding = process_glove_line(line, dim)
            if embedding is not None:
                model[word] = embedding
        return model

glove = load_glove_model("glove.840B.300d.txt", 300)

In [111]:
model = load_model('BiLSTM_QQP.h5')